In [4]:
import os
import pickle
import numpy as np
from tqdm import tqdm
import skimage.measure
from skimage.morphology import dilation, erosion
from sklearn.mixture import GaussianMixture
from skimage.measure import label
from PIL import Image
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
from ipywidgets import interactive
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
"""
Training GMM w/ background data
"""

downsampleSize = 2
skipAmount = 4
nComponents = 5
folderList = ['dataset/Crowd_PETS09/S0/Background/View_001/Time_13-06',
              'dataset/Crowd_PETS09/S0/Background/View_001/Time_13-32']

# ref = Image.open(folderList[0] + '/' + os.listdir(folderList[0])[0])
trainingData = [] # np.zeros(np.array(ref.convert('L')).shape)

for folder in folderList:
    for i, file in enumerate(os.listdir(folder)):
        if i % skipAmount == 0:
            img = Image.open(folder + '/' + file)
            img = np.array(img.convert('L')) # grayscale
            img = skimage.measure.block_reduce(img, downsampleSize, np.mean) # downsample
            trainingData.append(img)
trainingData = np.array(trainingData)

# N = np.arange(1, 6)
mixtureModels = []
for i in tqdm(range(trainingData.shape[1])):
    temp = []
    for j in range(trainingData.shape[2]):
        pixelValues = trainingData[:, i, j]
        # models = [None for j in range(len(N))]

        # for k in range(len(N)):
        #     models[k] = GaussianMixture(n_components=N[k]).fit(pixelValues.reshape(-1, 1))
        # AIC = [m.aic(pixelValues.reshape(-1, 1)) for m in models]
        # temp.append(models[np.argmin(AIC)])
        temp.append(GaussianMixture(n_components=nComponents).fit(pixelValues.reshape(-1, 1)))
    mixtureModels.append(temp)

with open('methods/baseline-indirect/GMM.pkl', 'wb') as f:
    pickle.dump(mixtureModels, f)

In [2]:
"""
Eval
"""
with open('methods/baseline-indirect/GMM.pkl', 'rb') as f:
    mixtureModels = pickle.load(f)

img = Image.open('dataset/Crowd_PETS09/S0/Regular_Flow/Time_13-57/View_001/frame_0018.jpg')
img = np.array(img.convert('L')) # grayscale
output = np.zeros(img.shape)
for i in tqdm(range(img.shape[0])):
    for j in range(img.shape[1]):
        model = mixtureModels[int(i / 2)][int(j / 2)]
        output[i, j] = model.score_samples(img[i, j].reshape(-1, 1))[0]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 576/576 [00:25<00:00, 22.82it/s]


In [19]:
"""
Visualization
"""
@widgets.interact(thresh=(-500, 0))
def plot_bin_val_distribution(thresh):
    print(f'Pixels under threshold: {np.sum(output < thresh)}')

    view = np.copy(output)
    t1 = np.where(view < thresh)
    t2 = np.where(view >= thresh)
    view[t1] = 255
    view[t2] = 0

    plt.imshow(img)
    plt.colorbar()
    plt.show()

    plt.imshow(view)
    plt.show()

    k = 5
    view = dilation(view, footprint=np.ones((k, k)))
    view = erosion(view, footprint=np.ones((k, k)))

    k = 7
    view = erosion(view, footprint=np.ones((k, k)))
    view = dilation(view, footprint=np.ones((k, k)))

    plt.imshow(view)
    plt.show()

    countComponents = label(view)
    print(f'Number of people: {np.max(countComponents)}')

interactive(children=(IntSlider(value=-250, description='thresh', max=0, min=-500), Output()), _dom_classes=('…

TODO: 
1) try adding mask on unimportant zones
2) do a search over threshold value and noise removal parameters
3) try cleaning result with gaussian blur from hw3
4) visualize good sample (pixel) of GMM distribution for presentation